# 검증세트

검증 세트(Validation Set)는 데이터 분석과 기계 학습에서 사용되는 중요한 개념입니다. 기계 학습 모델을 훈련할 때, 데이터는 보통 훈련 세트, 검증 세트, 그리고 테스트 세트로 나눠집니다.

- 훈련 세트: 모델이 학습하고 매개변수를 조정하는 데 사용됩니다.

- 검증 세트: 모델의 성능을 평가하고, 모델의 매개변수를 조정하는 데 사용됩니다. 이를 통해 모델이 과적합되지 않도록 합니다.

- 테스트 세트: 최종 모델의 성능을 평가하고, 모델이 실제 데이터에 얼마나 잘 적용되는지 확인하는 데 사용됩니다.

In [46]:
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [47]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [48]:
from sklearn.model_selection import train_test_split

#train / test set 분류
train_input, test_input, train_target, test_target = train_test_split(data, target,test_size = 0.2, random_state=42)

In [62]:
# 1차분류된 train_input, train_target을 0.2만큼 분류하여 검증세트 얻기
# 검증 세트 비율을 0.2로 설정
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [64]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [66]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차검증

교차 검증(Cross-Validation)은 머신 러닝 모델의 성능을 평가하고 일반화 능력을 검증하는 데 사용되는 기술입니다. 데이터를 여러 번 나누어 모델을 학습하고 평가하는 과정을 반복함으로써 모델의 안정성과 신뢰성을 높일 수 있습니다. 대표적인 교차 검증 기법으로는 K-겹 교차 검증(K-Fold Cross-Validation)이 있습니다.

K-겹 교차 검증(K-Fold Cross-Validation)
데이터 분할: 전체 데이터를 K개의 부분으로 나눕니다.

모델 학습과 평가:

- K번 반복해서 학습을 수행합니다.

- 각 반복에서는 K개의 부분 중 하나를 검증 세트로 사용하고, 나머지 K-1개를 훈련 세트로 사용합니다.

- 평균 성능 계산: 각 반복에서 얻은 성능을 평균하여 최종 성능을 평가합니다.

예를 들어, 5-겹 교차 검증을 하는 경우, 데이터는 5개의 부분으로 나뉘고, 각각의 부분이 한 번씩 검증 세트로 사용되며, 나머지 부분은 훈련 세트로 사용됩니다. 이렇게 5번의 학습과 평가를 수행한 후, 그 결과를 평균하여 모델의 성능을 평가합니다.

*장점*

- 데이터의 최대한 많은 부분을 사용하여 학습과 평가를 할 수 있습니다.

- 모델의 과적합(overfitting)을 방지할 수 있습니다.

- 모델의 성능을 보다 신뢰성 있게 평가할 수 있습니다.

*단점*
- 계산 비용이 많이 들 수 있습니다.

- 데이터가 너무 적을 경우, 여전히 일반화가 어려울 수 있습니다.


In [70]:
from sklearn.model_selection import cross_validate

# defalut는 5-폴드 교차검정
# train data를 5등분한 뒤 4개의 셋으로 훈련, 나머지 1개의 셋으로 정밀도 검증을 한다.
# 따라서 test_data만 따로 빼고 val_data는 따로 빼지 않음
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01031351, 0.03054905, 0.01001143, 0.01452947, 0.01111722]), 'score_time': array([0.00381184, 0.00942969, 0.        , 0.        , 0.00201416]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [82]:
import numpy as np

# 여기서의 test_score는 우리가 사용하던 val_input, val_target에 대한거지 진짜 test_input, test_target에 대한거 아님
# 위 코드의 cross_validate의 함수로 반환된 딕셔너리의 키가 'test_score'라서 그렇게 쓴것 오해하지말것 
print(np.mean(scores['test_score']))

0.855300214703487


우리는 앞서 train_test_split을 할 때 데이터를 섞어 주었기에 문제가 되지 않지만 만약 그렇지 않을때, 데이터를 섞어주어야 할때에는 아래와 같은 함수를 사용한다.

- 회귀 : KFold 분할기

- 분류 : StratifiedKFold

In [85]:
#위의 코드와 동일함
# StratifiedKFold에 매개변수 설정을 하지않았으므로 데이터를 섞지않음
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [91]:
# 만약 10-fold 교차검증을 하고 싶을 땐
splitter = StratifiedKFold(n_splits=10, shuffle= True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## Grid Search

하이퍼파라미터의 탐색과 교차검증을 동시에 수행하도록 도와주는 도구

Grid Search의 과정
하이퍼파라미터 설정: 여러 하이퍼파라미터와 각 하이퍼파라미터의 후보 값을 정의합니다.

모델 학습 및 평가: 각 하이퍼파라미터 조합에 대해 교차 검증을 통해 모델을 학습하고 평가합니다.

최적 조합 선택: 가장 높은 성능을 내는 하이퍼파라미터 조합을 선택합니다.

- 장점
    1. 다양한 하이퍼파라미터 조합을 시도하여 최적의 조합을 찾을 수 있습니다.

    2. 교차 검증을 통해 모델의 성능을 더욱 신뢰성 있게 평가할 수 있습니다.

- 단점
    1. 많은 하이퍼파라미터 조합을 시도해야 하기 때문에 계산 비용이 많이 들 수 있습니다.

In [108]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [110]:
# n_jobs 기본값 = 1 
# -1로 설정시 시스템의 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [112]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [114]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [116]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [120]:
#첫번째 값(0.0001)의 점수가 가장 높아 best_estimator의 param으로 선정되었다.
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [124]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [128]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5,20,1), 'min_samples_split': range(2,100,10)}

In [130]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

C:\Users\kdm12\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [146]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [148]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

매개변수의 값을 직접 설정하지 않고 매개변수를 샘플링 할 수 있는 확률분포 객체를 전달함.

randint는 랜덤 정수값들 리턴하고 uniform은 랜덤 실수값을 리턴함

In [150]:
from scipy.stats import uniform, randint

In [155]:
rgen = randint(0,10)
rgen.rvs(10)

array([8, 3, 0, 7, 7, 5, 7, 4, 3, 4], dtype=int64)

In [157]:
# 랜덤 정수값을 리턴하지만 이때 각 값들의 분포를 고르게 리턴함
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 94, 106, 101,  96,  97, 108,  99, 108,  85, 106], dtype=int64))

In [160]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.69409293, 0.42840576, 0.12534727, 0.62037284, 0.48528465,
       0.44451778, 0.00502321, 0.8327006 , 0.24878957, 0.16164296])

In [168]:
parmas = {'min_impurity_decrease' : uniform(0.0001, 0.001), 'max_depth' : randint(2,25), 'min_sample_split':randint(1,25), 'min_samples_leaf' : randint(1,25), }

In [170]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter= 100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                                        'min_samples_split': range(2, 100, 10)},
                   random_state=42)

In [172]:
print(gs.best_params_)

{'min_samples_split': 12, 'min_impurity_decrease': 0.0005, 'max_depth': 11}


In [176]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681935292811135


In [178]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
